<a href="https://colab.research.google.com/github/kjspring/stress-detection-wearable-devices/blob/main/modeling_WESAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load pickled data

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os

#! cd /content/drive/MyDrive/stress-prediction/ # Main file directory for this notebook on Google Drive
os.chdir('/content/drive/MyDrive/stress-prediction')
# absolute path of data directory
PATH = os.path.join(os.path.abspath(os.getcwd()), 'data') # Path of data folder on Google Drive

import joblib
data = joblib.load(f"{PATH}/pickle/WESAD_data_model.pickle") # read pickle file
labels = joblib.load(f"{PATH}/pickle/WESAD_labels_model.pickle") # read pickle file

Mounted at /content/drive


In [8]:
# Subsample the data and features
subsample_rate = 175  # subsample rate, e.g. to reduce from 700Hz to 4Hz

subsampled_data = data[::subsample_rate]
subsampled_labels = labels[::subsample_rate]

print(len(data))  # 2742499
print(len(subsampled_data))  # 15672

2742499
15672


In [9]:
# Try with a Datagenerator to reduce the RAM use
from keras.preprocessing.sequence import TimeseriesGenerator
random_state = 42
Hz = 4
sampling_rate = 5 # keep one data point out of 5
duration = 1 # how many minutes in the future the target after the end of the sequence
sequence_length = Hz * 60 * duration # observations will go back duration minutues
delay = sampling_rate*(sequence_length + duration*60*Hz - 1) # the target for a
                                                             # sequence will be
                                                             # duration (min)
                                                             # after the end of
                                                             # the sequence
batch_size = 1
shuffle = True

# Train Test Split
from sklearn.model_selection import train_test_split

X_dat, X_val, y_dat, y_val = train_test_split(subsampled_data, subsampled_labels, 
                                                 test_size = 0.2,
                                                 random_state=random_state)

X_train, X_test, y_train, y_test = train_test_split(X_dat, y_dat,
                                                    test_size = 0.2,
                                                    random_state = random_state)

train_data_gen = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=batch_size)
val_data_gen = TimeseriesGenerator(X_val, y_val, length=sequence_length, batch_size=batch_size)
test_data_gen = TimeseriesGenerator(X_test, y_test, length=sequence_length, batch_size=batch_size)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Define the LSTM model
model = Sequential()
model.add(LSTM(64, 
               activation='relu', 
               stateful = True, # To save RAM use
               batch_input_shape=(batch_size, sequence_length, 1)))
              # input_shape=(sequence_length, 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['binary_accuracy'])

# Train the model using the TimeSeriesGenerator
# train the model
epochs = 10
total_epochs = 0
for i in range(epochs):
    for j in range(len(train_data_gen)):
      print('Epoch', total_epochs+1, '/', epochs)
      model.fit(train_data_gen, epochs=1, validation_data=val_data_gen, verbose=1, shuffle=False)
      model.reset_states() # Need to reset since using LSTM stateful
      total_epochs += 1

Epoch 1 / 10
9789/9789 [==============================] - 984s 100ms/step - loss: nan - binary_accuracy: 0.8142 - val_loss: nan - val_binary_accuracy: 0.8138
Epoch 1 / 10
4513/9789 [============>.................] - ETA: 8:07 - loss: nan - binary_accuracy: 0.8043

In [ ]:
# Evaluate the model on the validation and test dataset

score = model.evaluate(val_data_gen)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])


test_loss = model.evaluate(test_data_gen)
print('Test loss:', test_loss)
print('Test accuracy:', score[1])